In [ ]:
!pip install momepy
!pip install numpy
!pip install osmnx
!pip install mapclassify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import momepy as mm
import libpysal
import pandas as pd
import matplotlib.pyplot as plt
import shapely
#from sympy import Point, Polygon
from shapely.geometry import Polygon, MultiPolygon
from shapely import wkt
from shapely.ops import linemerge, unary_union, polygonize

In [ ]:
def pt_in_poly(points, polygons):
    import numpy as np
    import pandas as pd

    this_idx, other_idx = points.sindex.query_bulk(polygons.geometry, predicate="contains")
    unique, counts = np.unique(this_idx, return_counts=True)
    indices = np.empty(len(polygons), dtype="object")
    indices[unique] = counts
    return pd.Series(indices, index=polygons.index)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

# Import data from step 1

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tess0 = gpd.read_file('/content/drive/MyDrive/Urban_preception/Milano_Analysis/1.2_Output/encl_tess0714.gpkg')

In [ ]:
tess0

,uID,eID,bID,nID,geometry
0,36494.0,7,0.0,5.0,"POLYGON Z ((515489.825 5039165.614 130.078, 51..."
1,36501.0,7,0.0,20465.0,"POLYGON Z ((515595.712 5039216.798 130.033, 51..."
2,39155.0,8,1.0,20563.0,"POLYGON Z ((515475.674 5038950.383 128.553, 51..."
3,39154.0,8,1.0,20302.0,"POLYGON Z ((515376.037 5039021.130 129.549, 51..."
4,39157.0,8,1.0,20563.0,"POLYGON Z ((515545.381 5039004.518 128.037, 51..."
...,...,...,...,...,...
63034,NaN,11182,NaN,NaN,"POLYGON Z ((512500.867 5028128.032 111.300, 51..."
63035,28633.0,11183,4192.0,16443.0,"POLYGON Z ((512889.439 5027339.968 111.300, 51..."
63036,NaN,11188,NaN,NaN,"POLYGON Z ((516113.326 5027955.780 107.496, 51..."
63037,16669.0,11191,4193.0,27553.0,"POLYGON Z ((520666.583 5034026.710 117.079, 52..."


In [ ]:
tess = tess0.dropna()

In [ ]:
tess

,uID,eID,bID,nID,geometry
0,36494.0,7,0.0,5.0,"POLYGON Z ((515489.825 5039165.614 130.078, 51..."
1,36501.0,7,0.0,20465.0,"POLYGON Z ((515595.712 5039216.798 130.033, 51..."
2,39155.0,8,1.0,20563.0,"POLYGON Z ((515475.674 5038950.383 128.553, 51..."
3,39154.0,8,1.0,20302.0,"POLYGON Z ((515376.037 5039021.130 129.549, 51..."
4,39157.0,8,1.0,20563.0,"POLYGON Z ((515545.381 5039004.518 128.037, 51..."
...,...,...,...,...,...
63032,6985.0,11177,4190.0,4691.0,"POLYGON Z ((510089.585 5030959.313 111.300, 51..."
63033,28644.0,11178,4191.0,13422.0,"POLYGON Z ((510299.428 5029667.818 111.300, 51..."
63035,28633.0,11183,4192.0,16443.0,"POLYGON Z ((512889.439 5027339.968 111.300, 51..."
63037,16669.0,11191,4193.0,27553.0,"POLYGON Z ((520666.583 5034026.710 117.079, 52..."


In [ ]:
buildings = gpd.read_file('/content/drive/MyDrive/Urban_preception/Milano_Analysis/1.2_Output/blgs0714.gpkg')

In [ ]:
invalid_geom = buildings[~buildings.is_valid]
print(f"Number of invalid geometries found: {invalid_geom.shape[0]}")

if invalid_geom.shape[0] > 0:
    buildings['geometry'] = buildings.buffer(0)

still_invalid_geom = buildings[~buildings.is_valid]
print(f"Number of invalid geometries after repair: {still_invalid_geom.shape[0]}")

Number of invalid geometries found: 0
Number of invalid geometries after repair: 0


In [ ]:
blocks = gpd.read_file('/content/drive/MyDrive/Urban_preception/Milano_Analysis/1.2_Output/blocks0714.gpkg')

In [ ]:
buildings['uID'] = buildings['uID'].astype(str)

In [ ]:
tess['uID'] = tess['uID'].astype(float).astype(int).astype(str)

In [ ]:
tess['tID'] = range(len(tess))

In [ ]:
tess

,uID,eID,bID,nID,geometry,tID
0,36494,7,0.0,5.0,"POLYGON Z ((515489.825 5039165.614 130.078, 51...",0
1,36501,7,0.0,20465.0,"POLYGON Z ((515595.712 5039216.798 130.033, 51...",1
2,39155,8,1.0,20563.0,"POLYGON Z ((515475.674 5038950.383 128.553, 51...",2
3,39154,8,1.0,20302.0,"POLYGON Z ((515376.037 5039021.130 129.549, 51...",3
4,39157,8,1.0,20563.0,"POLYGON Z ((515545.381 5039004.518 128.037, 51...",4
...,...,...,...,...,...,...
63032,6985,11177,4190.0,4691.0,"POLYGON Z ((510089.585 5030959.313 111.300, 51...",56057
63033,28644,11178,4191.0,13422.0,"POLYGON Z ((510299.428 5029667.818 111.300, 51...",56058
63035,28633,11183,4192.0,16443.0,"POLYGON Z ((512889.439 5027339.968 111.300, 51...",56059
63037,16669,11191,4193.0,27553.0,"POLYGON Z ((520666.583 5034026.710 117.079, 52...",56060


In [ ]:
buildings

,mean_height,SD_height,variance_height,count,uID,bID,nID,geometry
0,13.713333,1.352931,1.830422,3,1,38.0,449,"POLYGON ((512258.675 5038647.560, 512300.108 5..."
1,4.990000,0.000000,0.000000,1,2,38.0,310,"POLYGON ((512360.577 5038693.209, 512371.960 5..."
2,12.196000,0.000000,0.000000,1,3,601.0,16713,"POLYGON ((513301.155 5038209.714, 513312.315 5..."
3,23.721286,5.755591,33.126831,7,4,2580.0,16856,"POLYGON ((513429.650 5038179.936, 513419.177 5..."
4,6.749000,0.051151,0.002616,5,5,601.0,16692,"POLYGON ((513236.630 5038304.510, 513248.039 5..."
...,...,...,...,...,...,...,...,...
56021,5.171000,0.000000,0.000000,1,56022,1911.0,9554,"POLYGON Z ((520593.680 5042776.440 137.660, 52..."
56022,7.139333,1.988753,3.955137,6,56023,1911.0,9955,"POLYGON Z ((520679.434 5042275.308 134.800, 52..."
56023,5.630000,0.000000,0.000000,1,56024,1911.0,9955,"POLYGON Z ((520660.330 5042263.650 134.800, 52..."
56024,3.363000,0.000000,0.000000,1,56025,1911.0,9955,"POLYGON Z ((520599.500 5042317.360 134.765, 52..."


In [ ]:
buildings.columns

Index(['mean_height', 'SD_height', 'variance_height', 'count', 'uID', 'bID',
       'nID', 'geometry'],
      dtype='object')

In [ ]:
tess = tess.rename_geometry("tessellation").merge(buildings[["uID", "geometry","mean_height","SD_height","variance_height","count"]].rename_geometry("buildings"), on="uID", how="left")

In [ ]:
blg = tess.set_geometry('buildings') #.dropna()

In [ ]:
blg['mean_height'] = blg.mean_height.fillna(3)

In [ ]:
blg.loc[blg["mean_height"] < 3, "mean_height"] = 3 #substitute everything below 3m with 3m of height

In [ ]:
blg = blg[blg['buildings'].notna()]

In [ ]:
tess[['uID','eID','bID','nID','tID','tessellation']].to_file('/content/drive/MyDrive/Urban_preception/Milano_Analysis/1.3_Output' + '/tess.gpkg', driver="GPKG")

In [ ]:
blg

,uID,eID,bID,nID,tessellation,tID,buildings,mean_height,SD_height,variance_height,count
0,2945,2,0.0,2.0,"POLYGON Z ((515489.825 5039165.614 130.078, 51...",0,"POLYGON Z ((515530.629 5039152.900 130.188, 51...",4.160000,0.000000,0.000000,1
1,2949,2,0.0,735.0,"POLYGON Z ((515595.712 5039216.798 130.033, 51...",1,"POLYGON Z ((515488.384 5039249.736 130.027, 51...",40.992727,14.579634,212.565722,11
2,4832,3,1.0,754.0,"POLYGON Z ((515475.099 5038951.801 128.553, 51...",2,"POLYGON Z ((515530.459 5039012.915 128.628, 51...",6.700000,2.856945,8.162137,4
3,4831,3,1.0,721.0,"POLYGON Z ((515376.037 5039021.130 129.549, 51...",3,"POLYGON Z ((515440.374 5038968.673 130.352, 51...",22.952000,0.000000,0.000000,1
4,4834,3,1.0,754.0,"POLYGON Z ((515544.910 5039006.037 128.037, 51...",4,"POLYGON Z ((515564.392 5039040.251 128.364, 51...",5.294000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
3255,2814,395,226.0,1100.0,"POLYGON Z ((516459.656 5039787.079 131.110, 51...",3255,"POLYGON Z ((516425.018 5039861.120 136.629, 51...",15.870000,10.556461,111.438867,7
3256,5701,400,227.0,1126.0,"POLYGON Z ((516594.698 5040483.780 133.004, 51...",3256,"POLYGON Z ((516734.273 5040629.974 134.534, 51...",13.910200,9.196646,84.578304,10
3257,2392,441,228.0,1337.0,"POLYGON Z ((517563.972 5040539.221 133.570, 51...",3257,"POLYGON Z ((517596.133 5040615.052 133.951, 51...",22.566667,8.123357,65.988933,3
3258,23,449,229.0,1355.0,"POLYGON Z ((517667.236 5039110.274 127.702, 51...",3258,"POLYGON Z ((517730.573 5039233.574 128.118, 51...",9.052000,3.499674,12.247720,5


## measure characters

In [ ]:
blg = blg.rename(columns={'mean_height': 'heights'})

In [ ]:
blg.columns

Index(['uID', 'eID', 'bID', 'nID', 'tessellation', 'tID', 'buildings',
       'heights', 'SD_height', 'variance_height', 'count'],
      dtype='object')

In [ ]:
nan_counts = blg.isnull().sum()
print(nan_counts)

uID                  0
eID                  0
bID                  0
nID                  0
tessellation         0
tID                  0
buildings            0
heights              0
SD_height            0
variance_height      0
count                0
sdbAre               0
sdbVol               0
sdbPer               0
sdbCoA               0
ssbVFR               0
ssbCCo               0
ssbCor               0
ssbSqu               0
ssbERI               0
ssbElo               0
surface              0
volume_two_thirds    0
ssbFoF               0
dtype: int64


In [ ]:
blg['sdbAre'] = mm.Area(blg).series
blg['sdbVol'] = mm.Volume(blg, 'heights', 'sdbAre').series #only if you have building height
blg['sdbPer'] = mm.Perimeter(blg).series
blg['sdbCoA'] = mm.CourtyardArea(blg, 'sdbAre').series
#blg['ssbFoF'] = mm.FormFactor(blg, 'sdbVol', 'sdbAre').series #only if you have building height
blg['ssbVFR'] = mm.VolumeFacadeRatio(blg, 'heights', 'sdbVol', 'sdbPer').series #only if you have building height
blg['ssbCCo'] = mm.CircularCompactness(blg, 'sdbAre').series
blg['ssbCor'] = mm.Corners(blg).series
blg['ssbSqu'] = mm.Squareness(blg).series
blg['ssbERI'] = mm.EquivalentRectangularIndex(blg, 'sdbAre', 'sdbPer').series
blg['ssbElo'] = mm.Elongation(blg).series

  0%|          | 0/3260 [00:00<?, ?it/s]

  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
blg['surface'] = (blg['sdbPer'] * blg['heights']) + blg['sdbAre']
blg['volume_two_thirds'] = blg['sdbVol'] ** (2.0/3.0)
blg['ssbFoF'] = blg['surface'] / blg['volume_two_thirds']

In [ ]:
blg.drop(['surface','volume_two_thirds'], axis=1, inplace=True)

In [ ]:
cencon = mm.CentroidCorners(blg)
blg['ssbCCM'] = cencon.mean
blg['ssbCCD'] = cencon.std

  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
blg['stbOri'] = mm.Orientation(blg).series

  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
tess['stcOri'] = mm.Orientation(tess).series

  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
#add this
blg['sdbLAL'] = mm.LongestAxisLength(blg).series

In [ ]:
#add these
blg['ssbCVX'] = mm.Convexity(blg).series
blg['ssbFRA'] = mm.FractalDimension(blg,'sdbAre','sdbPer').series
blg['ssbSIN'] = mm.ShapeIndex(blg,longest_axis='sdbLAL',areas='sdbAre').series
blg['ssbSCO'] = mm.SquareCompactness(blg).series

In [ ]:
tess = tess.merge(blg[['tID', 'stbOri']], on='tID', how='left')

In [ ]:
tess['stbCeA'] = (tess['stbOri'] - tess['stcOri']).abs()
tess['sdcLAL'] = mm.LongestAxisLength(tess).series
tess['sdcAre'] = mm.Area(tess).series
tess['sscCCo'] = mm.CircularCompactness(tess, 'sdcAre').series
tess['sscERI'] = mm.EquivalentRectangularIndex(tess, 'sdcAre').series
tess['sicCAR'] = tess.buildings.area / tess['sdcAre']
#tess['sicFAR'] = (tess.buildings.area*(tess.height/3)) / tess['sdcAre']

In [ ]:
tess = tess.rename(columns={'mean_height': 'heights'})

In [ ]:
tess['sicFAR'] = mm.AreaRatio(tess.set_geometry('tessellation'), tess.set_geometry('buildings'), 'sdcAre', (tess.buildings.area*tess['heights'])/3, 'tID').series

In [ ]:
blg["mtbSWR"] = mm.SharedWallsRatio(blg).series

# Add queen_1

In [ ]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tess, ids="tID", geom_col = 'tessellation')

In [ ]:
tess["mtbAli"] = mm.Alignment(tess.set_geometry("buildings"), queen_1, "tID", "stbOri").series
tess["mtbNDi"] = mm.NeighborDistance(tess.set_geometry("buildings"), queen_1, "tID").series
tess["mtcWNe"] = mm.Neighbors(tess, queen_1, "tID", weighted=True).series
tess["mdcAre"] = mm.CoveredArea(tess, queen_1, "tID").series

  0%|          | 0/3260 [00:00<?, ?it/s]

  0%|          | 0/3260 [00:00<?, ?it/s]

  0%|          | 0/3260 [00:00<?, ?it/s]

  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
blg_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blg)

In [ ]:
blg_q1

In [ ]:
blg["libNCo"] = mm.Courtyards(blg, blg_q1).series
blg["ldbPWL"] = mm.PerimeterWall(blg, blg_q1).series

  0%|          | 0/3260 [00:00<?, ?it/s]

  0%|          | 0/3260 [00:00<?, ?it/s]

  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
blocks["ldkAre"] = mm.Area(blocks).series
blocks["ldkPer"] = mm.Perimeter(blocks).series
blocks["lskCCo"] = mm.CircularCompactness(blocks, "ldkAre").series
blocks["lskERI"] = mm.EquivalentRectangularIndex(blocks, "ldkAre", "ldkPer").series
blocks["lskCWA"] = mm.CompactnessWeightedAxis(blocks, "ldkAre", "ldkPer").series
blocks["ltkOri"] = mm.Orientation(blocks).series

  0%|          | 0/231 [00:00<?, ?it/s]

In [ ]:
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID")

blocks["ltkWNB"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True).series
blocks["likWBB"] = mm.Count(blocks, blg, "bID", "bID", weighted=True).series

  0%|          | 0/231 [00:00<?, ?it/s]

Save data to parquets

In [ ]:
path = '/content/drive/MyDrive/Urban_preception/Bicocca_Analysis/1.3_Output'

In [ ]:
tess.drop(columns=['tessellation', 'buildings']).to_parquet(path + '/tess_data.parquet')
blg.drop(columns=['tessellation', 'buildings']).to_parquet(path + '/blg_data.parquet')
blocks.drop(columns='geometry').to_parquet(path + '/blocks_data.parquet')

In [ ]:
queen3 = mm.sw_high(k=3, weights=queen_1)
queen1 = queen_1

In [ ]:
blg_queen = blg_q1

In [ ]:
blg

,uID,eID,bID,nID,tessellation,tID,buildings,heights,SD_height,variance_height,...,stbOri,mtbSWR,libNCo,ldbPWL,ltbIBD,ltcBuA,stbSAl,nodeID,libALM,libNeD
0,2945,2,0.0,2.0,"POLYGON Z ((515489.825 5039165.614 130.078, 51...",0,"POLYGON Z ((515530.629 5039152.900 130.188, 51...",4.160000,0.000000,0.000000,...,27.023689,0.000000e+00,0,20.740776,33.927896,0.882353,8.087914,2,NaN,NaN
1,2949,2,0.0,735.0,"POLYGON Z ((515595.712 5039216.798 130.033, 51...",1,"POLYGON Z ((515488.384 5039249.736 130.027, 51...",40.992727,14.579634,212.565722,...,44.521353,1.428150e-16,0,199.001324,35.138251,0.900000,23.865111,409,NaN,NaN
2,4832,3,1.0,754.0,"POLYGON Z ((515475.099 5038951.801 128.553, 51...",2,"POLYGON Z ((515530.459 5039012.915 128.628, 51...",6.700000,2.856945,8.162137,...,20.586367,2.052591e-01,0,318.165229,34.745671,0.766667,1.156656,709,NaN,NaN
3,4831,3,1.0,721.0,"POLYGON Z ((515376.037 5039021.130 129.549, 51...",3,"POLYGON Z ((515440.374 5038968.673 130.352, 51...",22.952000,0.000000,0.000000,...,18.593145,0.000000e+00,0,152.318615,37.326435,0.723404,1.524156,697,NaN,NaN
4,4834,3,1.0,754.0,"POLYGON Z ((515544.910 5039006.037 128.037, 51...",4,"POLYGON Z ((515564.392 5039040.251 128.364, 51...",5.294000,0.000000,0.000000,...,21.834187,-1.982157e-16,0,71.756700,38.014029,0.789474,0.091164,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,2814,395,226.0,1100.0,"POLYGON Z ((516459.656 5039787.079 131.110, 51...",3255,"POLYGON Z ((516425.018 5039861.120 136.629, 51...",15.870000,10.556461,111.438867,...,21.513125,1.047760e+00,3,648.401828,21.370824,0.800000,0.188132,278,NaN,NaN
3256,5701,400,227.0,1126.0,"POLYGON Z ((516594.698 5040483.780 133.004, 51...",3256,"POLYGON Z ((516734.273 5040629.974 134.534, 51...",13.910200,9.196646,84.578304,...,21.540830,1.605739e-16,5,496.088354,64.651988,0.844444,0.220100,12,NaN,NaN
3257,2392,441,228.0,1337.0,"POLYGON Z ((517563.972 5040539.221 133.570, 51...",3257,"POLYGON Z ((517596.133 5040615.052 133.951, 51...",22.566667,8.123357,65.988933,...,11.890227,-1.242984e-16,0,228.711230,12.250232,0.557692,0.162372,964,NaN,NaN
3258,23,449,229.0,1355.0,"POLYGON Z ((517667.236 5039110.274 127.702, 51...",3258,"POLYGON Z ((517730.573 5039233.574 128.118, 51...",9.052000,3.499674,12.247720,...,7.881577,0.000000e+00,0,514.581894,17.683017,0.523256,2.163688,958,NaN,NaN


In [ ]:
blg['ltbIBD'] = mm.MeanInterbuildingDistance(tess.set_geometry("buildings"), queen1, 'tID').series

Computing mean interbuilding distances...


  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
blg['ltcBuA'] = mm.BuildingAdjacency(tess.set_geometry("buildings"), queen3, 'tID').series

Calculating spatial weights...
Spatial weights ready...


Calculating adjacency:   0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
tess['licGDe'] = mm.Density(tess.set_geometry('tessellation'), (tess.buildings.area*tess['heights'])/3, queen3, 'tID', 'sdcAre').series

  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
tess['ltcWRB'] = mm.BlocksCount(tess.set_geometry("tessellation"), 'bID', queen3, 'tID').series

  0%|          | 0/3260 [00:00<?, ?it/s]

In [ ]:
tess

,uID,eID,bID,nID,tessellation,tID,buildings,heights,SD_height,variance_height,...,sicCAR,sicFAR,mtbAli,mtbNDi,mtcWNe,mdcAre,licGDe,ltcWRB,stcSAl,nodeID
0,2945,2,0.0,2.0,"POLYGON Z ((515489.825 5039165.614 130.078, 51...",0,"POLYGON Z ((515530.629 5039152.900 130.188, 51...",4.160000,0.000000,0.000000,...,0.002631,0.003649,12.932336,60.532695,0.007041,33731.439142,0.973553,0.000038,2.073291e-01,2
1,2949,2,0.0,735.0,"POLYGON Z ((515595.712 5039216.798 130.033, 51...",1,"POLYGON Z ((515488.384 5039249.736 130.027, 51...",40.992727,14.579634,212.565722,...,0.120928,1.652387,23.647286,50.535377,0.008231,34050.161594,0.906352,0.000034,1.824500e+00,409
2,4832,3,1.0,754.0,"POLYGON Z ((515475.099 5038951.801 128.553, 51...",2,"POLYGON Z ((515530.459 5039012.915 128.628, 51...",6.700000,2.856945,8.162137,...,0.146175,0.326457,2.474673,43.653767,0.012418,32472.226116,0.986414,0.000081,7.052545e+00,709
3,4831,3,1.0,721.0,"POLYGON Z ((515376.037 5039021.130 129.549, 51...",3,"POLYGON Z ((515440.374 5038968.673 130.352, 51...",22.952000,0.000000,0.000000,...,0.054093,0.413850,2.710310,62.380425,0.014205,49268.135736,0.897889,0.000063,8.302988e+00,697
4,4834,3,1.0,754.0,"POLYGON Z ((515544.910 5039006.037 128.037, 51...",4,"POLYGON Z ((515564.392 5039040.251 128.364, 51...",5.294000,0.000000,0.000000,...,0.214843,0.379127,3.524690,13.230126,0.017916,22316.573737,0.902054,0.000072,1.578367e-03,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,2814,395,226.0,1100.0,"POLYGON Z ((516459.656 5039787.079 131.110, 51...",3255,"POLYGON Z ((516425.018 5039861.120 136.629, 51...",15.870000,10.556461,111.438867,...,0.372424,1.970122,0.058638,15.707415,0.006314,24980.163845,2.453613,0.000093,3.595346e-11,278
3256,5701,400,227.0,1126.0,"POLYGON Z ((516594.698 5040483.780 133.004, 51...",3256,"POLYGON Z ((516734.273 5040629.974 134.534, 51...",13.910200,9.196646,84.578304,...,0.463555,2.149382,0.562092,60.920758,0.005919,132015.223867,1.030718,0.000026,1.034138e-01,12
3257,2392,441,228.0,1337.0,"POLYGON Z ((517563.972 5040539.221 133.570, 51...",3257,"POLYGON Z ((517596.133 5040615.052 133.951, 51...",22.566667,8.123357,65.988933,...,0.306200,2.303307,0.098972,24.185567,0.022162,18399.504569,1.534324,0.000156,2.190866e-04,964
3258,23,449,229.0,1355.0,"POLYGON Z ((517667.236 5039110.274 127.702, 51...",3258,"POLYGON Z ((517730.573 5039233.574 128.118, 51...",9.052000,3.499674,12.247720,...,0.259927,0.784286,6.278081,25.821943,0.025689,35913.958371,1.061597,0.000106,6.032727e-01,958


Save data to parquets and spatial weights matrices to gal files

In [ ]:
tess.drop(columns=['tessellation', 'buildings']).to_parquet(path + '/tess_data.parquet')
blg.drop(columns=['tessellation', 'buildings']).to_parquet(path + '/blg_data.parquet')

In [ ]:
fo = libpysal.io.open(path + '/AMSqueen1.gal', 'w')
fo.write(queen1)
fo.close()

fo = libpysal.io.open(path + '/AMSqueen3.gal', 'w')
fo.write(queen3)
fo.close()

fo = libpysal.io.open(path + '/AMSblg_queen.gal', 'w')
fo.write(blg_queen)
fo.close()

# Add street

In [ ]:
streets = gpd.read_file('/content/drive/MyDrive/Urban_preception/Bicocca_Analysis/1.2_Output/streets.gpkg')

In [ ]:
streets["sdsLen"] = mm.Perimeter(streets).series
tess["stcSAl"] = mm.StreetAlignment(tess, streets, "stcOri", "nID").series
blg["stbSAl"] = mm.StreetAlignment(blg, streets, "stbOri", "nID").series

In [ ]:
# Add street continuity
continuity = mm.COINS(streets)
stroke_gdf = continuity.stroke_gdf()
streets["sssCtn"] = continuity.stroke_attribute()

In [ ]:

profile = mm.StreetProfile(streets, tess.set_geometry("buildings"), heights=tess.heights)
streets["sdsSPW"] = profile.w
streets["sdsSPH"] = profile.h #only if you have building height
streets["sdsSPR"] = profile.p #only if you have building height
streets["sdsSPO"] = profile.o
streets["sdsSWD"] = profile.wd
streets["sdsSHD"] = profile.hd #only if you have building height

streets["sssLin"] = mm.Linearity(streets).series
streets["sdsAre"] = mm.Reached(streets, tess, "nID", "nID", mode="sum", values="sdcAre").series
streets["sisBpM"] = mm.Count(streets, blg, "nID", "nID", weighted=True).series

  0%|          | 0/1523 [00:00<?, ?it/s]

In [ ]:
# Add NeighboringStreetOrientationDeviation
streets['sssNso'] = mm.NeighboringStreetOrientationDeviation(streets).series

In [ ]:
tess.drop(columns=['tessellation', 'buildings']).to_parquet(path + '/tess_data.parquet')
blg.drop(columns=['tessellation', 'buildings']).to_parquet(path + '/blg_data.parquet')
streets.drop(columns='geometry').to_parquet(path + '/streets_data.parquet')

In [ ]:
str_q1 = libpysal.weights.contiguity.Queen.from_dataframe(streets)

streets["misRea"] = mm.Reached(
    streets, tess, "nID", "nID", spatial_weights=str_q1, mode="count"
).series
streets["mdsAre"] = mm.Reached(streets, tess, "nID", "nID", spatial_weights=str_q1,
                               mode="sum").series

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

In [ ]:
graph = mm.gdf_to_nx(streets)

print("node degree")
graph = mm.node_degree(graph)

print("subgraph")
graph = mm.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
)
print("cds length")
graph = mm.cds_length(graph, radius=3, name="ldsCDL")

print("clustering")
graph = mm.clustering(graph, name="xcnSCl")

print("mean_node_dist")
graph = mm.mean_node_dist(graph, name="mtdMDi")

nodes, edges, sw = mm.nx_to_gdf(graph, spatial_weights=True)

node degree
subgraph


  0%|          | 0/1064 [00:00<?, ?it/s]

cds length


  0%|          | 0/1064 [00:00<?, ?it/s]

clustering
mean_node_dist


  0%|          | 0/1064 [00:00<?, ?it/s]

In [ ]:
print("saving")
nodes.to_file(path + '/nodes.gpkg', driver="GPKG")
edges.to_file(path + '/egdes.gpkg', driver="GPKG")

saving


In [ ]:
fo = libpysal.io.open(path + '/UAPnodes.gal', "w")
fo.write(sw)
fo.close()

edges_w3 = mm.sw_high(k=3, gdf=edges)
edges["ldsMSL"] = mm.SegmentsLength(edges, spatial_weights=edges_w3, mean=True).series

edges["ldsRea"] = mm.Reached(edges, tess, "nID", "nID", spatial_weights=edges_w3).series
edges["ldsRea"] = mm.Reached(
    edges, tess, "nID", "nID", spatial_weights=edges_w3, mode="sum", values="sdcAre"
).series

nodes_w5 = mm.sw_high(k=5, weights=sw)
nodes["lddNDe"] = mm.NodeDensity(nodes, edges, nodes_w5).series
nodes["linWID"] = mm.NodeDensity(
    nodes, edges, nodes_w5, weighted=True, node_degree="degree"
).series

blg["nodeID"] = mm.get_node_id(blg, nodes, edges, "nodeID", "nID")
tess = tess.merge(blg[["tID", "nodeID"]], on="tID", how="left")

nodes_w3 = mm.sw_high(k=3, weights=sw)

nodes["lddRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", nodes_w3).series
nodes["lddARe"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", nodes_w3, mode="sum", values="sdcAre"
).series

nodes["sddAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", mode="sum", values="sdcAre"
).series
nodes["midRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", spatial_weights=sw).series
nodes["midAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", spatial_weights=sw, mode="sum", values="sdcAre"
).series

nodes.rename(
    columns={
        "degree": "mtdDeg",
        "meshedness": "lcdMes",
        "local_closeness": "lcnClo",
        "proportion_3": "linP3W",
        "proportion_4": "linP4W",
        "proportion_0": "linPDE",
    }, inplace=True
)

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1064 [00:00<?, ?it/s]

  0%|          | 0/1064 [00:00<?, ?it/s]

...

  0%|          | 0/1064 [00:00<?, ?it/s]

  0%|          | 0/1064 [00:00<?, ?it/s]

  0%|          | 0/1064 [00:00<?, ?it/s]

  0%|          | 0/1064 [00:00<?, ?it/s]

  0%|          | 0/1064 [00:00<?, ?it/s]

In [ ]:
tess.drop(columns=['tessellation', 'buildings']).to_parquet(path + '/tess_data.parquet')
blg.drop(columns=['tessellation', 'buildings']).to_parquet(path + '/blg_data.parquet')
nodes.drop(columns='geometry').to_parquet(path + '/nodes_data.parquet')
edges.drop(columns='geometry').to_parquet(path + '/edges_data.parquet')

In [ ]:
blocks.to_file('blocks_result.gpkg', driver='GPKG')
!cp 'blocks_result.gpkg' '/content/drive/MyDrive/Urban_preception/Bicocca_Analysis/1.3_Output'

In [ ]:
blocks.drop(columns='geometry').to_parquet(path + '/blocks_result.parquet')

In [ ]:
blocks.columns

Index(['bID', 'geometry', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWBB'],
      dtype='object')

In [ ]:
tess.columns

Index(['uID', 'eID', 'bID', 'nID', 'tessellation', 'tID', 'buildings',
       'heights', 'SD_height', 'variance_height', 'count', 'stcOri', 'stbOri',
       'stbCeA', 'sdcLAL', 'sdcAre', 'sscCCo', 'sscERI', 'sicCAR', 'sicFAR',
       'mtbAli', 'mtbNDi', 'mtcWNe', 'mdcAre', 'licGDe', 'ltcWRB', 'stcSAl',
       'nodeID'],
      dtype='object')

In [ ]:
blg.columns

Index(['uID', 'eID', 'bID', 'nID', 'tessellation', 'tID', 'buildings',
       'heights', 'SD_height', 'variance_height', 'count', 'sdbAre', 'sdbVol',
       'sdbPer', 'sdbCoA', 'ssbVFR', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbERI',
       'ssbElo', 'ssbFoF', 'ssbCCM', 'ssbCCD', 'stbOri', 'mtbSWR', 'libNCo',
       'ldbPWL', 'ltbIBD', 'ltcBuA', 'stbSAl', 'nodeID'],
      dtype='object')

In [ ]:
streets.columns

Index(['STRATO', 'TEMA', 'CLASSE', 'ID_ZRIL', 'FEATURE_ID', 'EL_STR_TY',
       'EL_STR_CF', 'EL_STR_STA', 'EL_STR_FON', 'EL_STR_CL', 'EL_STR_SED',
       'EL_STR_LIV', 'COD_VIA', 'COD_VIA2', 'COD_ESTESA', 'VERS_CT10', 'width',
       'nID', 'geometry', 'sdsLen', 'sdsSPW', 'sdsSPH', 'sdsSPR', 'sdsSPO',
       'sdsSWD', 'sdsSHD', 'sssLin', 'sdsAre', 'sisBpM', 'misRea', 'mdsAre'],
      dtype='object')

In [ ]:
tess.drop(columns=['tessellation', 'buildings']).to_csv('/content/drive/MyDrive/Urban_preception/Bicocca_Analysis/1.3_Output/tess_result.csv', index=False)

In [ ]:
blg.drop(columns=['tessellation', 'buildings']).to_csv('/content/drive/MyDrive/Urban_preception/Bicocca_Analysis/1.3_Output/blg_result.csv', index=False)

In [ ]:
streets.to_file('streets_result.gpkg', driver='GPKG')
!cp 'streets_result.gpkg' '/content/drive/MyDrive/Urban_preception/Bicocca_Analysis/1.3_Output'

Merge

In [ ]:
merged = tess.merge(blg.drop(columns=['uID','eID','bID','nID','tessellation','buildings','heights','stbOri','nodeID']), on='tID', how='left')

In [ ]:
merged = merged.merge(blocks.drop(columns='geometry'), on='bID', how='left')
merged = merged.merge(edges.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')

In [ ]:
merged.rename(columns={'heights': 'sdbHei'}, inplace=True)

In [ ]:
print(list(merged.columns))

['uID', 'eID', 'bID', 'nID', 'tessellation', 'tID', 'buildings', 'sdbHei', 'SD_height_x', 'variance_height_x', 'count_x', 'stcOri', 'stbOri', 'stbCeA', 'sdcLAL', 'sdcAre', 'sscCCo', 'sscERI', 'sicCAR', 'sicFAR', 'mtbAli', 'mtbNDi', 'mtcWNe', 'mdcAre', 'licGDe', 'ltcWRB', 'stcSAl', 'nodeID', 'SD_height_y', 'variance_height_y', 'count_y', 'sdbAre', 'sdbVol', 'sdbPer', 'sdbCoA', 'ssbVFR', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbERI', 'ssbElo', 'ssbFoF', 'ssbCCM', 'ssbCCD', 'mtbSWR', 'libNCo', 'ldbPWL', 'ltbIBD', 'ltcBuA', 'stbSAl', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB', 'likWBB', 'STRATO', 'TEMA', 'CLASSE', 'ID_ZRIL', 'FEATURE_ID', 'EL_STR_TY', 'EL_STR_CF', 'EL_STR_STA', 'EL_STR_FON', 'EL_STR_CL', 'EL_STR_SED', 'EL_STR_LIV', 'COD_VIA', 'COD_VIA2', 'COD_ESTESA', 'VERS_CT10', 'width', 'sdsLen', 'sdsSPW', 'sdsSPH', 'sdsSPR', 'sdsSPO', 'sdsSWD', 'sdsSHD', 'sssLin', 'sdsAre', 'sisBpM', 'misRea', 'mdsAre', 'mm_len', 'cdsbool', 'node_start', 'node_end', 'ldsMSL', 'ldsRea

In [ ]:
merged

,uID,eID,bID,nID,tessellation,tID,buildings,sdbHei,SD_height_x,variance_height_x,...,ldsCDL,xcnSCl,mtdMDi,lddNDe,linWID,lddRea,lddARe,sddAre,midRea,midAre
0,2945,2,0.0,2.0,"POLYGON Z ((515489.825 5039165.614 130.078, 51...",0,"POLYGON Z ((515530.629 5039152.900 130.188, 51...",4.160000,0.000000,0.000000,...,146.652676,0.000000,182.331536,0.006792,0.015122,30,136176.598973,24001.160571,10,42665.581606
1,2949,2,0.0,735.0,"POLYGON Z ((515595.712 5039216.798 130.033, 51...",1,"POLYGON Z ((515488.384 5039249.736 130.027, 51...",40.992727,14.579634,212.565722,...,415.941632,0.000000,86.244343,0.007047,0.015744,30,139969.987499,15759.553094,5,39557.592527
2,4832,3,1.0,754.0,"POLYGON Z ((515475.099 5038951.801 128.553, 51...",2,"POLYGON Z ((515530.459 5039012.915 128.628, 51...",6.700000,2.856945,8.162137,...,0.000000,0.000000,163.638433,0.007007,0.015337,27,126235.849960,10483.125769,10,36458.593852
3,4831,3,1.0,721.0,"POLYGON Z ((515376.037 5039021.130 129.549, 51...",3,"POLYGON Z ((515440.374 5038968.673 130.352, 51...",22.952000,0.000000,0.000000,...,244.706080,0.090909,92.506634,0.009014,0.018420,35,147714.317479,16814.622008,9,37883.110912
4,4834,3,1.0,754.0,"POLYGON Z ((515544.910 5039006.037 128.037, 51...",4,"POLYGON Z ((515564.392 5039040.251 128.364, 51...",5.294000,0.000000,0.000000,...,146.652676,0.000000,182.331536,0.006792,0.015122,30,136176.598973,24001.160571,10,42665.581606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,2814,395,226.0,1100.0,"POLYGON Z ((516459.656 5039787.079 131.110, 51...",3255,"POLYGON Z ((516425.018 5039861.120 136.629, 51...",15.870000,10.556461,111.438867,...,0.000000,0.031250,103.724722,0.008127,0.018849,35,186988.938624,11781.334892,4,31923.694450
3256,5701,400,227.0,1126.0,"POLYGON Z ((516594.698 5040483.780 133.004, 51...",3256,"POLYGON Z ((516734.273 5040629.974 134.534, 51...",13.910200,9.196646,84.578304,...,0.000000,0.076923,187.311932,0.006318,0.015143,25,249833.034887,37530.269908,5,66549.941929
3257,2392,441,228.0,1337.0,"POLYGON Z ((517563.972 5040539.221 133.570, 51...",3257,"POLYGON Z ((517596.133 5040615.052 133.951, 51...",22.566667,8.123357,65.988933,...,0.000000,0.166667,64.989508,0.007448,0.017544,105,122303.198693,4128.795124,12,14674.062949
3258,23,449,229.0,1355.0,"POLYGON Z ((517667.236 5039110.274 127.702, 51...",3258,"POLYGON Z ((517730.573 5039233.574 128.118, 51...",9.052000,3.499674,12.247720,...,0.000000,0.111111,98.508492,0.007609,0.016636,170,246097.366260,19550.950429,31,57754.227409


In [ ]:
merged.drop(columns=['tessellation', 'buildings']).to_csv('/content/drive/MyDrive/Urban_preception/Bicocca_Analysis/1.3_Output/merged_result.csv', index=False)

Clean columns to keep only data

In [ ]:
primary = merged.drop(columns=['uID', 'eID', 'bID', 'nID', 'tessellation', 'buildings', 'mm_len', 'cdsbool',
                               'nodeID', 'node_start', 'node_end'])

In [ ]:
primary.to_parquet(path + '/primary.parquet')

In [ ]:
primary

,tID,sdbHei,SD_height_x,variance_height_x,count_x,stcOri,stbOri,stbCeA,sdcLAL,sdcAre,...,ldsCDL,xcnSCl,mtdMDi,lddNDe,linWID,lddRea,lddARe,sddAre,midRea,midAre
0,0,4.160000,0.000000,0.000000,1,19.143104,27.023689,7.880585,173.963881,10109.313234,...,146.652676,0.000000,182.331536,0.006792,0.015122,30,136176.598973,24001.160571,10,42665.581606
1,1,40.992727,14.579634,212.565722,11,18.831741,44.521353,25.689612,197.308039,10197.468136,...,415.941632,0.000000,86.244343,0.007047,0.015744,30,139969.987499,15759.553094,5,39557.592527
2,2,6.700000,2.856945,8.162137,4,14.690478,20.586367,5.895889,160.319181,8424.726400,...,0.000000,0.000000,163.638433,0.007007,0.015337,27,126235.849960,10483.125769,10,36458.593852
3,3,22.952000,0.000000,0.000000,1,11.814313,18.593145,6.778832,186.428359,14411.871457,...,244.706080,0.090909,92.506634,0.009014,0.018420,35,147714.317479,16814.622008,9,37883.110912
4,4,5.294000,0.000000,0.000000,1,21.744602,21.834187,0.089586,73.891694,1218.256465,...,146.652676,0.000000,182.331536,0.006792,0.015122,30,136176.598973,24001.160571,10,42665.581606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,3255,15.870000,10.556461,111.438867,7,21.324993,21.513125,0.188132,182.029616,10312.377254,...,0.000000,0.031250,103.724722,0.008127,0.018849,35,186988.938624,11781.334892,4,31923.694450
3256,3256,13.910200,9.196646,84.578304,10,21.424143,21.540830,0.116686,247.282029,25304.297263,...,0.000000,0.076923,187.311932,0.006318,0.015143,25,249833.034887,37530.269908,5,66549.941929
3257,3257,22.566667,8.123357,65.988933,3,11.728074,11.890227,0.162153,100.468332,4128.795124,...,0.000000,0.166667,64.989508,0.007448,0.017544,105,122303.198693,4128.795124,12,14674.062949
3258,3258,9.052000,3.499674,12.247720,5,6.321162,7.881577,1.560415,161.241319,11221.431203,...,0.000000,0.111111,98.508492,0.007609,0.016636,170,246097.366260,19550.950429,31,57754.227409


In [ ]:
print(primary.dtypes)

tID                    int64
sdbHei               float64
SD_height_x          float64
variance_height_x    float64
count_x                int64
                      ...   
lddRea                 int64
lddARe               float64
sddAre               float64
midRea                 int64
midAre               float64
Length: 105, dtype: object


In [ ]:
print(list(primary.columns))

['tID', 'sdbHei', 'SD_height_x', 'variance_height_x', 'count_x', 'stcOri', 'stbOri', 'stbCeA', 'sdcLAL', 'sdcAre', 'sscCCo', 'sscERI', 'sicCAR', 'sicFAR', 'mtbAli', 'mtbNDi', 'mtcWNe', 'mdcAre', 'licGDe', 'ltcWRB', 'stcSAl', 'SD_height_y', 'variance_height_y', 'count_y', 'sdbAre', 'sdbVol', 'sdbPer', 'sdbCoA', 'ssbVFR', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbERI', 'ssbElo', 'ssbFoF', 'ssbCCM', 'ssbCCD', 'ssbCVX', 'ssbFRA', 'sdbLAL', 'ssbSIN', 'ssbSCO', 'mtbSWR', 'libNCo', 'ldbPWL', 'libALM', 'ltbIBD', 'ltcBuA', 'libNeD', 'stbSAl', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB', 'likWBB', 'STRATO', 'TEMA', 'CLASSE', 'ID_ZRIL', 'FEATURE_ID', 'EL_STR_TY', 'EL_STR_CF', 'EL_STR_STA', 'EL_STR_FON', 'EL_STR_CL', 'EL_STR_SED', 'EL_STR_LIV', 'COD_VIA', 'COD_VIA2', 'COD_ESTESA', 'VERS_CT10', 'width', 'sdsLen', 'sdsSPW', 'sdsSPH', 'sdsSPR', 'sdsSPO', 'sdsSWD', 'sdsSHD', 'sssLin', 'sdsAre', 'sisBpM', 'misRea', 'mdsAre', 'ldsMSL', 'ldsRea', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', '